In [1]:
import pandas as pd
import math
from datetime import date
import glob
import json
import numpy as np
import nbimporter
import sys
import numpy as np


In [2]:
def load_data(file):
    daily = pd.read_csv(file)
    if 'Date' in daily.columns:
        daily['Date'] = pd.to_datetime(daily['Date'])
    else:
        daily['Date'] = pd.to_datetime(daily['Datetime'], utc=True).dt.tz_convert('US/Eastern')
    daily = daily.sort_values(by='Date', ascending=True)
    daily['Day'] = daily['Date'].dt.strftime('%m/%d/%Y')
    daily['Time'] = daily['Date'].dt.strftime('%H:%M')
    daily = daily.dropna().reset_index(drop=True)
    return daily

def load_yahoo_data(file):
    daily = pd.read_csv(file)
    if 'Date' in daily.columns:
        daily['Date'] = pd.to_datetime(daily['Date'])
    else:
        daily['Date'] = pd.to_datetime(daily['Datetime'], utc=True).dt.tz_convert('US/Eastern')
    daily = daily.sort_values(by='Date', ascending=True)
    daily['Day'] = daily['Date'].dt.strftime('%Y-%m-%d')
    daily['Time'] = daily['Date'].dt.strftime('%H:%M')
    daily = daily.dropna().reset_index(drop=True)
    return daily
def load_NKE_data():
    files = ["/Users/partha/Downloads/0-DTE/Stocks/NKE-20Year.csv",
            
            ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df

def load_stock_data(files):
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df

def load_stocks_data():
    files = ["/Users/partha/Downloads/0-DTE/stocks/GOOG-Dec-16.csv",
#              "/Users/partha/Downloads/0-DTE/indexes/NFLX-Dec-16.csv",
#              "/Users/partha/Downloads/0-DTE/indexes/GOOG-Dec-16.csv"
            ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
        df
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df
def load_all_5min_spx():
    files = [ 
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-13.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-16.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-12.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-9.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-8.csv",
            "/Users/partha/Downloads/0-DTE/indexes/SPX-DEC-6.csv",
             "/Users/partha/Downloads/0-DTE/indexes/SPX-DEC-7.csv",
            "/Users/partha/Downloads/0-DTE/indexes/spx-oct-28-snapshot.csv",
        "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-20.csv",
    "/Users/partha/Downloads/0-DTE/indexes/SPX-Dec-19.csv"]

    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df[df['Day'] > '08/31/2022']
    return df.drop_duplicates().sort_values(by='Date', ascending=True)
# df = load_all_5min_spx()
# df.head()

def load_5min_vix():
    files = [ 
        "/Users/partha/Downloads/0-DTE/indexes/VIX-Dec-9.csv",
        "/Users/partha/Downloads/0-DTE/indexes/VIX-Nov-09.csv",
        "/Users/partha/Downloads/0-DTE/indexes/VIX-Nov-09.csv"
    ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df[df['Day'] > '08/31/2022']
    return df.drop_duplicates().sort_values(by='Date', ascending=True)
    
def load_annual_data():
    files = ["/Users/partha/Downloads/0-DTE/indexes/SPX-20Year.csv",
            
            ]
    df = pd.DataFrame()
    for file in files:
        print ('Loading file {}'.format(file))
        df = df.append(load_data(file))
    df = df.drop_duplicates().sort_values(by='Date', ascending=True)
    return df


In [3]:
def build_seq(segment, split_size=10):
    
    rge = segment['High'].max() - segment['Low'].min()
    ilist = np.array_split(segment, split_size)
    amap = []
   
    for idf in ilist:
        try:
            idf = idf.reset_index(drop=True)
            s = int((idf[:1]['Open'].min()-segment['Low'].min())*split_size/rge)
            e = int((idf.iloc[-1]['Close'].min()-segment['Low'].min())*split_size/rge)
            if not amap:
                amap.append(s)
            else:    
                amap.append(e)
        except Exception as e:
            print ("Exception while building Sequence {}".format(e))
            pass    
    return amap

def train_segments(chart, seq_map, split_size=10):
    chart = chart.reset_index(drop=True)
    for i in range(0, len(chart)):
        segment = chart[i:i+split_size]
        segment = segment.reset_index(drop=True)
        if len(segment) < split_size:
            continue
        
        amap = build_seq(segment,split_size)
        
        if len(amap) != split_size:
               continue
        if i % 200 == 0:
            
            #print ('Completed {} iterations'.format(i))
            pass
        amap = [item - amap[0] for item in amap]
        
        seq_map[str(amap)] = {"Sequence": amap, 
                              'Start date': segment.iloc[0]['Day'], 
                              'End Date': segment.iloc[-1]['Day'],
                              'Start Time' : segment.iloc[0]['Time'],
                              'End Time' : segment.iloc[-1]['Time']
                             }              
    return seq_map

class Progress:
    
    def __init__(self, total):
        self.total = total
        self.count = 0
        
    def completed(self):
        self.count += 1
        print ("Completed {}/{}".format(self.count, self.total))

In [4]:
sym = 'ABBV'
def train_symbol(sym):
    df = load_yahoo_data('/Users/partha/Downloads/0-DTE/Stocks/sp500/{}-20Year.csv'.format(sym))
    df = df[df['Date'] <= '2022-01-01']
    model = train_segments(df, {}, 20)
    f = open('/Users/partha/Downloads/0-DTE/Stocks/sp500-models-Upto2021/{}.json'.format(sym), 'w')
    f.write(json.dumps(model))
    f.close()
    #print ('Model for {} saved successfully'.format(sym))

In [8]:
# Train hourly data
def train_hourly():
    df = load_data('/Users/partha/Downloads/0-DTE/indexes/hourly/Hourly-SPX-Jan-05.csv')
    #train=df[:1000]
    #test = df[-399:]
    train = df
    model = train_segments(train, {}, 20)
    f = open('/Users/partha/Downloads/0-DTE/models/SPX-hourly-Jan05.json'.format(sym), 'w')
    f.write(json.dumps(model))
    f.close()
    print ("Model file saved successfully")
train_hourly()


/opt/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname ET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Model file saved successfully


In [19]:
symbols = pd.read_csv('Symbols.csv')['Symbol']
prg = Progress(len(symbols))

for sym in symbols:
    try:
        train_symbol(sym)
    except:
        pass
    finally:
        prg.completed()

Completed 1/505
Completed 2/505
Completed 3/505
Completed 4/505
Completed 5/505
Completed 6/505
Completed 7/505
Completed 8/505
Completed 9/505
Completed 10/505
Completed 11/505
Completed 12/505
Completed 13/505
Completed 14/505
Completed 15/505
Completed 16/505
Completed 17/505
Completed 18/505
Completed 19/505
Completed 20/505
Completed 21/505
Completed 22/505
Completed 23/505
Completed 24/505
Completed 25/505
Completed 26/505
Completed 27/505
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence flo

Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequen

Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequen

Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequen

Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequen

...

Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequen

Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequen

Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequence float division by zero
Exception while building Sequen

Completed 28/505
Completed 29/505
Completed 30/505
Completed 31/505
Completed 32/505
Completed 33/505
Completed 34/505
Completed 35/505
Completed 36/505
Completed 37/505
Completed 38/505
Completed 39/505
Completed 40/505
Completed 41/505
Completed 42/505
Completed 43/505
Completed 44/505
Completed 45/505
Completed 46/505
Completed 47/505
Completed 48/505
Completed 49/505
Completed 50/505
Completed 51/505
Completed 52/505
Completed 53/505
Completed 54/505
Completed 55/505
Completed 56/505
Completed 57/505
Completed 58/505
Completed 59/505
Completed 60/505
Completed 61/505
Completed 62/505
Completed 63/505
Completed 64/505
Completed 65/505
Completed 66/505
Completed 67/505
Completed 68/505
Completed 69/505
Completed 70/505
Completed 71/505
Completed 72/505
Completed 73/505
Completed 74/505
Completed 75/505
Completed 76/505
Completed 77/505
Completed 78/505
Completed 79/505
Completed 80/505
Completed 81/505
Completed 82/505
Completed 83/505
Completed 84/505
Completed 85/505
Completed 86/5

Completed 488/505
Completed 489/505
Completed 490/505
Completed 491/505
Completed 492/505
Completed 493/505
Completed 494/505
Completed 495/505
Completed 496/505
Completed 497/505
Completed 498/505
Completed 499/505
Completed 500/505
Completed 501/505
Completed 502/505
Completed 503/505
Completed 504/505
Completed 505/505


In [17]:
prg = Progress(5)
prg.completed()

Completed 1/5
